In [9]:
import numpy as np


#count sort
def countsort(A,EXP):
    C=np.zeros(10,dtype=int)
    n=len(A)
    B=np.zeros(n,dtype=int)
    digits=(A//EXP)%10
    for i in digits:
        #index=(i//EXP)%10
        C[i]+=1
    for j in range (1,10):
        C[j]+=C[j-1]
    for m in range (n-1,-1,-1):
        B[C[digits[m]]-1]=A[m]
        C[digits[m]]-=1
    return B

#radix sort
def radixsort_core(A):
    if not isinstance(A, np.ndarray):
        A = np.array(A, dtype=int)
    maxV=np.max(A)
    exp=1
    while maxV//exp>0:
        A=countsort(A,exp)
        exp*=10   ####
    return A
   

#对负数和小数的处理(预处理和后处理)
def radixsort_neg(A,max_decimal):
    neg = [-x for x in A if x < 0]       # 负数取相反数
    pos = [x for x in A if x >= 0]       # 非负数
  
    neg_sorted = radixsort_core(neg)[::-1] if len(neg)>0 else []  # 负数排序后反转
    pos_sorted = radixsort_core(pos) if len(pos)>0 else []
    A=[-x for x in neg_sorted] + list(pos_sorted)
    J=0
    for i in A:
        A[J]=i/10**max_decimal
        J+=1
    return A
#对小数的预处理
def redixsort(A):
    A=np.array(A,dtype=float)
    neg_inf=np.isneginf(A)
    pos_inf=np.isposinf(A)
    finite=np.isfinite(A)
    neg_inf_v=A[neg_inf]
    pos_inf_v=A[pos_inf]
    A=A[finite]
    max_decimal=0
    for num in A:
        if (num % 1)!=0:
            decimal=str(num).split('.')[1]
            max_decimal=max(max_decimal,len(decimal))
    if max_decimal>0:
        A*=10**max_decimal
        print(A)
    A_sorted=np.concatenate([neg_inf_v,radixsort_neg(A,max_decimal),pos_inf_v])
    return A_sorted 
    #radixsort_neg(A,max_decimal)


In [10]:
A= [float('inf'), float('-inf'),123, -456, 78.9, -0.123, 45, -67.89, 0, 3.14159]
redixsort(A)

[ 1.23000e+07 -4.56000e+07  7.89000e+06 -1.23000e+04  4.50000e+06
 -6.78900e+06  0.00000e+00  3.14159e+05]


array([        -inf, -4.56000e+02, -6.78900e+01, -1.23000e-01,
        0.00000e+00,  3.14159e+00,  4.50000e+01,  7.89000e+01,
        1.23000e+02,          inf])

In [36]:
import numpy as np
#很好的测试，暴露了函数的很多问题，待改进2（已经包含9），5，7，10
# 复杂的测试用例集合
test_cases = [
    # 1. 混合正负整数和小数
    [123, -456, 78.9, -0.123, 45, -67.89, 0, 3.14159],
    
    # 2. 极值测试
    [float('inf'), float('-inf'), 1e10, -1e10, 0, 1e-10, -1e-10],
    
    # 3. 大量重复元素
    [1.5, 1.5, 1.5, 2.3, 2.3, -4.7, -4.7, -4.7, -4.7, 0, 0, 0],
    
    # 4. 不同小数位数混合
    [0.1, 0.01, 0.001, 0.0001, 0.12345, 1.23456, 12.3456, 123.456],
    
    # 5. 边界值测试
    [999.999, -999.999, 1000.001, -1000.001, 0.000001, -0.000001],
    
    # 6. 大范围数值
    [1, 10, 100, 1000, 10000, 100000, 0.1, 0.01, 0.001, 0.0001],
    
    # 7. 特殊小数情况
    [1/3, 1/7, 2/3, np.pi, np.e, -np.pi, -np.e],
    
    # 8. 几乎相等的数值
    [1.000001, 1.000002, 1.000003, 0.999999, 0.999998, -1.000001, -1.000002],
    
    # 9. 科学计数法表示的数值
    [1e-5, 2e-5, 1e5, 2e5, -1e-5, -2e-5, -1e5, -2e5],
    
    # 10. 随机复杂案例
    [np.random.uniform(-1000, 1000) for _ in range(50)] + 
    [np.random.randint(-1000, 1000) for _ in range(50)]
]
def comprehensive_test():
    """全面测试基数排序算法"""
    
    for i, test_case in enumerate(test_cases):
        print(f"\n=== 测试用例 {i+1} ===")
        print(f"原数组 (前10个): {test_case[:10]}{'...' if len(test_case) > 10 else ''}")
        
        try:
            # 使用我们的基数排序
            our_result = radixsort(test_case.copy())
            
            # 使用Python内置排序作为基准
            python_result = sorted(test_case)
            
            # 检查结果是否一致（考虑浮点数精度）
            is_correct = True
            for our_val, py_val in zip(our_result, python_result):
                if abs(our_val - py_val) > 1e-10:  # 允许的误差范围
                    is_correct = False
                    break
            
            print(f"我们的排序 (前10个): {our_result[:10]}{'...' if len(our_result) > 10 else ''}")
            print(f"Python排序 (前10个): {python_result[:10]}{'...' if len(python_result) > 10 else ''}")
            print(f"结果一致: {is_correct}")
            
            if not is_correct:
                print("!!! 排序结果不一致 !!!")
                # 找出第一个不一致的位置
                for idx, (our_val, py_val) in enumerate(zip(our_result, python_result)):
                    if abs(our_val - py_val) > 1e-10:
                        print(f"第一个不一致位置: {idx}, 我们的值: {our_val}, Python的值: {py_val}")
                        break
        
        except Exception as e:
            print(f"排序过程中出错: {e}")
            import traceback
            traceback.print_exc()

# 运行测试
comprehensive_test()


=== 测试用例 1 ===
原数组 (前10个): [123, -456, 78.9, -0.123, 45, -67.89, 0, 3.14159]
我们的排序 (前10个): [   0    0    3   45   78  123 -456  -67]
Python排序 (前10个): [-456, -67.89, -0.123, 0, 3.14159, 45, 78.9, 123]
结果一致: False
!!! 排序结果不一致 !!!
第一个不一致位置: 0, 我们的值: 0, Python的值: -456

=== 测试用例 2 ===
原数组 (前10个): [inf, -inf, 10000000000.0, -10000000000.0, 0, 1e-10, -1e-10]
排序过程中出错: cannot convert float infinity to integer

=== 测试用例 3 ===
原数组 (前10个): [1.5, 1.5, 1.5, 2.3, 2.3, -4.7, -4.7, -4.7, -4.7, 0]...
我们的排序 (前10个): [ 0  0  0  1  1  1  2  2 -4 -4]...
Python排序 (前10个): [-4.7, -4.7, -4.7, -4.7, 0, 0, 0, 1.5, 1.5, 1.5]...
结果一致: False
!!! 排序结果不一致 !!!
第一个不一致位置: 0, 我们的值: 0, Python的值: -4.7

=== 测试用例 4 ===
原数组 (前10个): [0.1, 0.01, 0.001, 0.0001, 0.12345, 1.23456, 12.3456, 123.456]
我们的排序 (前10个): [  0   0   0   0   0   1  12 123]
Python排序 (前10个): [0.0001, 0.001, 0.01, 0.1, 0.12345, 1.23456, 12.3456, 123.456]
结果一致: False
!!! 排序结果不一致 !!!
第一个不一致位置: 0, 我们的值: 0, Python的值: 0.0001

=== 测试用例 5 ===
原数组 (前10个): [999.999, -999

Traceback (most recent call last):
  File "C:\Users\yuqin\AppData\Local\Temp\ipykernel_29204\4253409302.py", line 45, in comprehensive_test
    our_result = radixsort(test_case.copy())
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\yuqin\AppData\Local\Temp\ipykernel_29204\4154003276.py", line 23, in radixsort
    A = np.array(A, dtype=int)
        ^^^^^^^^^^^^^^^^^^^^^^
OverflowError: cannot convert float infinity to integer


In [24]:
num=456
decimal=str(num).split('.')[1]
print(decimal)

IndexError: list index out of range